In [ ]:
# CLEAN START 
%cd /kaggle/working

# Clone OpenFace repo
!git clone https://github.com/TadasBaltrusaitis/OpenFace.git
%cd OpenFace

# Download pre-trained models (required for FeatureExtraction)
!bash download_models.sh

# Install dependencies
!apt-get update -y
!apt-get install -y build-essential cmake libopenblas-dev liblapack-dev \
    libboost-all-dev libx11-dev libpng-dev libjpeg-dev

# Clone and build dlib (C++ version)
!git clone https://github.com/davisking/dlib.git
%cd dlib
!mkdir build
%cd build
!cmake ..
!cmake --build . --config Release
!make install
%cd /kaggle/working/OpenFace

# Build OpenFace (this is the important corrected step!)
!mkdir build
%cd build
!cmake ..
!make -j4
%cd /kaggle/working

In [ ]:
!ls -l /kaggle/working/OpenFace/build/bin/

In [ ]:
#!/usr/bin/env python3
# openface_gazelle_hybrid_with_logs_full.py
# Full pipeline: OpenFace (face bbox CSVs) + GazeLLE (heatmaps + in/out)
# Produces feature matrices, per-frame logs, partner/elsewhere events, and annotated videos.
#
# Edit the VIDEO* / OUTDIR / OPENFACE_BIN paths below as needed for your Kaggle environment.

import os
import sys
import csv
import math
import time
import glob
from typing import Optional, List, Tuple, Dict, Any

import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageFont
import imageio.v3 as iio
import matplotlib.cm as cm
import torch

# ---------------- USER PATHS (edit) ----------------
VIDEO1 = "/kaggle/input/project-videos/project_videos/p_17/camera1_output.mp4"
VIDEO2 = "/kaggle/input/project-videos/project_videos/p_17/camera2_output.mp4"
OUTDIR = "/kaggle/working/gaze_outputs"
OPENFACE_BIN = "/kaggle/working/OpenFace/build/bin/FeatureExtraction"
# --------------------------------------------------

# ---------------- PARAMETERS (tweak) ----------------
INOUT_THRESH = 0.5
SMOOTH_WINDOW = 3
FIXATION_RADIUS_NORM = 0.03
MOTION_THRESH_NORM = 0.02
WORKSPACE_Y_MIN = 0.6
PARTNER_BOX_EXPAND = 1.5
JOINT_GAZE_DISTANCE_NORM = 0.05
HEATMAP_IOU_THRESH = 0.05
# --------------------------------------------------

# ---------------- logging & font -------------------
def log(msg: str):
    print(f"[{time.strftime('%H:%M:%S')}] {msg}", flush=True)

def get_font(px=14):
    try:
        return ImageFont.truetype("DejaVuSans.ttf", px)
    except Exception:
        try:
            return ImageFont.truetype("arial.ttf", px)
        except Exception:
            return ImageFont.load_default()

FONT = get_font(14)
# --------------------------------------------------

# ----------------- helpers -------------------------
def run_openface(video_path: str, outdir: str):
    os.makedirs(outdir, exist_ok=True)
    cmd = f'{OPENFACE_BIN} -f "{video_path}" -out_dir "{outdir}" -2Dfp -pose -gaze'
    log(f"Running OpenFace (may take a while): {cmd}")
    rc = os.system(cmd)
    if rc != 0:
        log(f"WARNING: OpenFace returned non-zero exit code ({rc}). If CSVs exist in outdir, you can continue.")

def find_openface_csv(video_path: str, outdir: str) -> Optional[str]:
    base = os.path.splitext(os.path.basename(video_path))[0]
    cands = sorted(glob.glob(os.path.join(outdir, "*.csv")))
    # prefer csv containing base name
    filtered = [p for p in cands if base in os.path.basename(p)]
    if filtered:
        return filtered[0]
    # else return most recently modified CSV
    return cands[0] if cands else None

COL_MAP = {
    "frame": ["frame", "frameNumber", "frame_num"],
    "face_x": ["face_x", "face_x_px", "face_xpos"],
    "face_y": ["face_y", "face_y_px", "face_ypos"],
    "face_w": ["face_w", "face_width", "face_x_size"],
    "face_h": ["face_h", "face_height", "face_y_size"],
    # fallback landmarks
    "lx": ["x_36", "landmark_36_x", "X_36"],
    "ly": ["y_36", "landmark_36_y", "Y_36"],
    "gax": ["gaze_angle_x", "GazeAngleX", "gaze_angle_x(deg)"],
    "gay": ["gaze_angle_y", "GazeAngleY", "gaze_angle_y(deg)"],
}
def find_col(df: pd.DataFrame, candidates: List[str]) -> Optional[str]:
    for c in candidates:
        if c in df.columns:
            return c
    return None

def detect_columns(df: pd.DataFrame) -> Dict[str, Optional[str]]:
    out = {}
    for k, cand in COL_MAP.items():
        out[k] = find_col(df, cand)
    return out

def norm_box_from_px(x, y, w, h, W, H):
    xmin = max(0.0, x) / float(W)
    ymin = max(0.0, y) / float(H)
    xmax = min(float(W), x + w) / float(W)
    ymax = min(float(H), y + h) / float(H)
    return [xmin, ymin, xmax, ymax]

def rotate_point_ccw90(pt: Tuple[float,float]) -> Optional[Tuple[float,float]]:
    if pt is None: return None
    x,y = pt
    xc = x - 0.5; yc = y - 0.5
    xr = -yc; yr = xc
    return (xr + 0.5, yr + 0.5)

def rotate_box_ccw90(box: Optional[List[float]]) -> Optional[List[float]]:
    if box is None: return None
    xmin,ymin,xmax,ymax = box
    corners = [(xmin,ymin),(xmin,ymax),(xmax,ymin),(xmax,ymax)]
    rc = [rotate_point_ccw90(c) for c in corners]
    xs = [c[0] for c in rc]; ys = [c[1] for c in rc]
    return [max(0.0,min(xs)), max(0.0,min(ys)), min(1.0,max(xs)), min(1.0,max(ys))]

def clamp01(v): return max(0.0, min(1.0, float(v)))

def moving_average_points(points: List[Optional[Tuple[float,float]]], window: int):
    if window <= 1: return points[:]
    n=len(points); out=[]
    half = window//2
    for i in range(n):
        s=max(0,i-half); e=min(n, i+half+1)
        xs=[]; ys=[]
        for j in range(s,e):
            p = points[j]
            if p is not None:
                xs.append(p[0]); ys.append(p[1])
        out.append((float(np.mean(xs)), float(np.mean(ys))) if xs else None)
    return out

def normalized_distance(p1, p2):
    return math.hypot(p1[0]-p2[0], p1[1]-p2[1])

def compute_fixations(points: List[Optional[Tuple[float,float]]], fps: float, radius_norm: float):
    fix=[]
    n=len(points); i=0
    while i<n:
        if points[i] is None:
            i+=1; continue
        s=i; pts=[points[i]]; j=i+1
        while j<n and points[j] is not None and normalized_distance(points[j], points[s]) <= radius_norm:
            pts.append(points[j]); j+=1
        e = j-1
        dur = (e - s + 1) / float(fps)
        centroid = (float(np.mean([p[0] for p in pts])), float(np.mean([p[1] for p in pts])))
        spread = float(np.mean([math.hypot(p[0]-centroid[0], p[1]-centroid[1]) for p in pts]))
        fix.append((s, e, dur, centroid, spread))
        i = j
    return fix

def compute_shift_freq(points: List[Optional[Tuple[float,float]]], fps: float, motion_thresh_norm: float):
    n=len(points)
    if n==0: return 0.0
    shifts=0; prev=None
    for p in points:
        if p is None:
            prev=None; continue
        if prev is not None and normalized_distance(prev,p) > motion_thresh_norm:
            shifts += 1
        prev = p
    secs = n / float(fps)
    return shifts / (secs + 1e-12)

def heatmap_to_rgba_img(hm, out_size, alpha=140):
    if hm is None:
        return None
    arr = hm if isinstance(hm, np.ndarray) else np.array(hm)
    # normalize
    arr = arr - arr.min()
    if arr.max() > 0:
        arr = arr / (arr.max() + 1e-12)
    else:
        arr = arr * 0.0
    img = Image.fromarray((arr * 255).astype(np.uint8))
    try:
        img = img.resize(out_size, resample=Image.Resampling.BILINEAR)
    except Exception:
        img = img.resize(out_size, resample=Image.BILINEAR)
    cmap = cm.get_cmap("jet")
    cmaped = cmap(np.array(img) / 255.0)
    rgba = (cmaped * 255).astype(np.uint8)
    rgba_img = Image.fromarray(rgba, mode="RGBA")
    rgba_img.putalpha(alpha)
    return rgba_img

def draw_overlay(pil: Image.Image, norm_bbox: Optional[List[float]], gaze_pt_norm: Optional[Tuple[float,float]],
                 hm: Optional[np.ndarray]=None, label: Optional[str]=None, color="lime"):
    w,h = pil.size
    out = pil.convert("RGBA").copy()
    draw = ImageDraw.Draw(out)
    if hm is not None:
        try:
            hm_rgba = heatmap_to_rgba_img(hm, (w,h), alpha=140)
            if hm_rgba is not None:
                out = Image.alpha_composite(out, hm_rgba)
                draw = ImageDraw.Draw(out)
        except Exception:
            pass
    if norm_bbox:
        rect = [norm_bbox[0]*w, norm_bbox[1]*h, norm_bbox[2]*w, norm_bbox[3]*h]
        draw.rectangle(rect, outline=color, width=max(1,int(min(w,h)*0.006)))
    if gaze_pt_norm is not None:
        gx = gaze_pt_norm[0]*w; gy = gaze_pt_norm[1]*h
        cx = ((norm_bbox[0] + norm_bbox[2]) / 2.0)*w if norm_bbox else w/2
        cy = ((norm_bbox[1] + norm_bbox[3]) / 2.0)*h if norm_bbox else h/2
        r = max(4, int(min(w,h)*0.008))
        draw.ellipse([(gx-r,gy-r),(gx+r,gy+r)], fill=color)
        draw.line([(cx,cy),(gx,gy)], fill=color, width=max(1,int(min(w,h)*0.006)))
    if label:
        draw.text((10,10), label, fill="white", font=FONT)
    return np.array(out.convert("RGB"))

def find_heatmap_peak_xy(hm: np.ndarray) -> Tuple[float,float,float]:
    # returns (gx_norm, gy_norm, peak_value)
    if hm is None:
        return (None, None, 0.0)
    H,W = hm.shape
    iy, ix = np.unravel_index(np.argmax(hm), hm.shape)
    gx = float(ix) / float(W)
    gy = float(iy) / float(H)
    return (gx, gy, float(hm[iy, ix]))

def classify_gaze_context(gaze_pt, partner_box, workspace_y_min):
    """
    Classifies gaze into:
    - looking_to_partner
    - looking_workspace
    - looking_elsewhere
    """
    if gaze_pt is None:
        return "looking_elsewhere"

    x, y = gaze_pt

    # ---------- Check partner region ----------
    if partner_box is not None:
        xmin, ymin, xmax, ymax = partner_box
        cx = (xmin + xmax) / 2.0
        cy = (ymin + ymax) / 2.0

        w = (xmax - xmin) * PARTNER_BOX_EXPAND
        h = (ymax - ymin) * PARTNER_BOX_EXPAND

        # expanded bounding box
        px_min = max(0.0, cx - w / 2.0)
        px_max = min(1.0, cx + w / 2.0)
        py_min = max(0.0, cy - h / 2.0)
        py_max = min(1.0, cy + h / 2.0)

        if px_min <= x <= px_max and py_min <= y <= py_max:
            return "looking_to_partner"

    # ---------- workspace region (bottom of screen) ----------
    if y >= workspace_y_min:
        return "looking_workspace"

    # ---------- otherwise ----------
    return "looking_elsewhere"
# --------------------------------------------------

# --------------- process one video ----------------
def process_video(video_path: str, of_csv: str, model, transform, device, label="P"):
    log(f"[{label}] loading frames from {video_path}")
    frames = list(iio.imiter(video_path, plugin="FFMPEG"))
    meta = iio.immeta(video_path, plugin="FFMPEG")
    fps = float(meta.get("fps", 25.0))
    if len(frames) == 0:
        raise RuntimeError(f"No frames read from {video_path}")
    H, W = frames[0].shape[0], frames[0].shape[1]

    df = pd.read_csv(of_csv)
    cols = detect_columns(df)

    recs = []  # each: dict with frame_idx, norm_bbox, gaze_pt, heatmap, inout, face_px bbox (optional)
    log(f"[{label}] processing {min(len(frames), len(df))} frames (csv rows vs frames)")
    n = min(len(frames), len(df))
    for i in range(n):
        row = df.iloc[i]
        # compute face bbox normalized
        nb = None
        try:
            if cols["face_x"] and cols["face_y"] and cols["face_w"] and cols["face_h"]:
                fx = float(row[cols["face_x"]]); fy = float(row[cols["face_y"]])
                fw = float(row[cols["face_w"]]); fh = float(row[cols["face_h"]])
                nb = norm_box_from_px(fx,fy,fw,fh,W,H)
        except Exception:
            nb = None
        # fallback to landmark
        if nb is None and cols["lx"] and cols["ly"]:
            try:
                lx = float(row[cols["lx"]]); ly = float(row[cols["ly"]])
                cx = lx / float(W); cy = ly / float(H)
                bw,bh = 0.2, 0.30
                nb = [clamp01(cx - bw/2), clamp01(cy - bh/2), clamp01(cx + bw/2), clamp01(cy + bh/2)]
            except Exception:
                nb = None

        pil = Image.fromarray(frames[i])
        gaze_pt = None; hm_np = None; inout_val = None; peak_val = 0.0
        if nb is not None:
            # call gazelle model
            with torch.no_grad():
                img_t = transform(pil).unsqueeze(0).to(device)
                inp = {"images": img_t, "bboxes": [[nb]]}
                out = model(inp)
            # heatmap extraction (handle list or tensor shape)
            hm = out.get("heatmap", None)
            if hm is None:
                hm_np = None
            else:
                # hm may be list or tensor: prefer hm[0][0] or hm[0,0]
                if isinstance(hm, list):
                    hm_t = hm[0][0]
                else:
                    # shape may be (B, C, H, W)
                    hm_t = hm[0,0]
                hm_np = hm_t.detach().cpu().numpy()
                gx, gy, peak_val = find_heatmap_peak_xy(hm_np)
                inout_arr = out.get("inout", None)
                try:
                    if inout_arr is not None:
                        # out["inout"][0] could be tensor shape (1,) or list
                        v = inout_arr[0]
                        inout_val = float(v.detach().cpu().numpy()) if hasattr(v, "detach") else float(v)
                except Exception:
                    inout_val = None
                # decide gaze point visible
                if inout_val is None or inout_val > INOUT_THRESH:
                    gaze_pt = (gx, gy)
                else:
                    gaze_pt = None

        recs.append({
            "frame_idx": i,
            "norm_bbox": nb,
            "gaze_pt": gaze_pt,
            "heatmap": hm_np,
            "inout": inout_val,
            # face pixel bbox saved for convenience
            "frame_bbox_px": None
        })

    return frames[:n], fps, recs

# --------- feature computation & outputs ----------
def compute_and_save_all(v1, v2, outdir):
    os.makedirs(outdir, exist_ok=True)

    # Ensure OpenFace CSVs exist (run OpenFace first)
    log("Running OpenFace to generate CSVs (if needed). This can be skipped if you already have CSVs.")
    run_openface(v1, outdir)
    run_openface(v2, outdir)
    csv1 = find_openface_csv(v1, outdir)
    csv2 = find_openface_csv(v2, outdir)
    if csv1 is None or csv2 is None:
        log("ERROR: Could not find OpenFace CSVs. Look in outdir and re-run.")
        log(f"Files in {outdir}: {sorted(glob.glob(os.path.join(outdir,'*')))}")
        sys.exit(1)
    log(f"Found OpenFace CSVs:\n P1 -> {csv1}\n P2 -> {csv2}")

    # Load model (GazeLLE) from torch.hub
    device = "cuda" if torch.cuda.is_available() else "cpu"
    log(f"Loading GazeLLE via torch.hub (device={device}) ...")
    model, transform = torch.hub.load("fkryan/gazelle", "gazelle_dinov2_vitl14_inout")
    model.to(device).eval()
    log("Model loaded.")

    # Process both videos
    frames1, fps1, rec1 = process_video(v1, csv1, model, transform, device, label="P1")
    frames2, fps2, rec2 = process_video(v2, csv2, model, transform, device, label="P2")
    n_frames = min(len(rec1), len(rec2))
    fps = min(float(fps1), float(fps2))
    log(f"Processing summary: frames P1={len(rec1)}, P2={len(rec2)} -> using n_frames={n_frames}, fps≈{fps:.2f}")

    # Extract smoothed gaze trajectories
    gp1 = [rec1[i]["gaze_pt"] for i in range(n_frames)]
    gp2 = [rec2[i]["gaze_pt"] for i in range(n_frames)]
    gp1_sm = moving_average_points(gp1, SMOOTH_WINDOW)
    gp2_sm = moving_average_points(gp2, SMOOTH_WINDOW)

    # Fixations and shifts
    fix1 = compute_fixations(gp1_sm, fps, FIXATION_RADIUS_NORM)
    fix2 = compute_fixations(gp2_sm, fps, FIXATION_RADIUS_NORM)
    avg_fix1 = float(np.mean([f[2] for f in fix1])) if len(fix1)>0 else 0.0
    avg_fix2 = float(np.mean([f[2] for f in fix2])) if len(fix2)>0 else 0.0
    num_fix1 = len(fix1); num_fix2 = len(fix2)
    avg_spread1 = float(np.mean([f[4] for f in fix1])) if len(fix1)>0 else 0.0
    avg_spread2 = float(np.mean([f[4] for f in fix2])) if len(fix2)>0 else 0.0
    shift_freq1 = compute_shift_freq(gp1_sm, fps, MOTION_THRESH_NORM)
    shift_freq2 = compute_shift_freq(gp2_sm, fps, MOTION_THRESH_NORM)

    # Align P2 into P1/world (rotate 90° CCW) for joint metrics
    gp2_al = [rotate_point_ccw90(p) if p is not None else None for p in gp2_sm]
    nb1 = [rec1[i]["norm_bbox"] for i in range(n_frames)]
    nb2 = [rec2[i]["norm_bbox"] for i in range(n_frames)]
    nb2_al = [rotate_box_ccw90(b) if b is not None else None for b in nb2]
    hm1 = [rec1[i]["heatmap"] for i in range(n_frames)]
    hm2_al = [np.rot90(rec2[i]["heatmap"], k=1) if rec2[i]["heatmap"] is not None else None for i in range(n_frames)]

    # Contexts
    ctx1 = [classify_gaze_context(gp1_sm[i], nb2_al[i], WORKSPACE_Y_MIN) for i in range(n_frames)]
    ctx2 = [classify_gaze_context(gp2_sm[i], None, WORKSPACE_Y_MIN) for i in range(n_frames)]

    # Joint frames: distance between gp1_sm and gp2_al <= threshold
    joint_flags = [ (gp1_sm[i] is not None and gp2_al[i] is not None and normalized_distance(gp1_sm[i], gp2_al[i]) <= JOINT_GAZE_DISTANCE_NORM) for i in range(n_frames) ]
    joint_frames = sum(1 for f in joint_flags if f)
    joint_time_sec = joint_frames / (fps + 1e-12)

    # Shared Attention IoU (heatmap overlap approx)
    iou_list = []
    for i in range(n_frames):
        a1 = hm1[i]; a2 = hm2_al[i]
        if a1 is None or a2 is None:
            iou_list.append(0.0); continue
        # thresholded binary maps
        t1 = a1.max() * HEATMAP_IOU_THRESH
        t2 = a2.max() * HEATMAP_IOU_THRESH
        b1 = (a1 >= t1).astype(np.uint8)
        b2 = (a2 >= t2).astype(np.uint8)
        inter = np.logical_and(b1,b2).sum()
        uni = np.logical_or(b1,b2).sum()
        iou_list.append(float(inter)/float(uni) if uni>0 else 0.0)
    mean_iou = float(np.mean(iou_list)) if len(iou_list)>0 else 0.0

    # episodes of shared focus
    episodes=[]
    i=0
    while i < n_frames:
        if joint_flags[i]:
            s=i; j=i+1
            while j<n_frames and joint_flags[j]: j+=1
            e=j-1
            episodes.append((s,e,(e-s+1)/fps))
            i=j
        else:
            i+=1
    num_episodes = len(episodes)
    avg_episode_dur = float(np.mean([ep[2] for ep in episodes])) if episodes else 0.0

    # Mutual gaze: both looking_to_partner
    mutual_frames = sum(1 for i in range(n_frames) if ctx1[i]=="looking_to_partner" and ctx2[i]=="looking_to_partner")
    mutual_time = mutual_frames / (fps + 1e-12)

    # percent times
    def pct(lst, tag): return float(lst.count(tag)) / float(n_frames) * 100.0
    pct_workspace1 = pct(ctx1, "looking_workspace"); pct_partner1 = pct(ctx1, "looking_to_partner"); pct_else1 = pct(ctx1, "looking_elsewhere")
    pct_workspace2 = pct(ctx2, "looking_workspace"); pct_partner2 = pct(ctx2, "looking_to_partner"); pct_else2 = pct(ctx2, "looking_elsewhere")

    # gaze direction stats (angle from face center to gaze point)
    def direction_angles(nb_list, gp_list):
        angs=[]
        for i in range(n_frames):
            nb = nb_list[i]; gp = gp_list[i]
            if nb is None or gp is None: continue
            cx = (nb[0]+nb[2])/2.0; cy = (nb[1]+nb[3])/2.0
            dx = gp[0] - cx; dy = gp[1] - cy
            ang = (math.degrees(math.atan2(dy, dx)) + 360.0) % 360.0
            angs.append(ang)
        return angs
    angs1 = direction_angles(nb1, gp1_sm)
    angs2 = direction_angles(nb2_al, gp2_al)
    avg_angle1 = float(np.mean(angs1)) if angs1 else 0.0
    std_angle1 = float(np.std(angs1)) if angs1 else 0.0
    avg_angle2 = float(np.mean(angs2)) if angs2 else 0.0
    std_angle2 = float(np.std(angs2)) if angs2 else 0.0

    # Task Engagement Score (TES) & Social Engagement Score (SES)
    # TES = %workspace / (%elsewhere + 1)  (keeps scale, avoid div by zero)
    TES1 = pct_workspace1 / (pct_else1 + 1.0)
    TES2 = pct_workspace2 / (pct_else2 + 1.0)
    # SES = %TimeLookingAtPartner / 100  (0..1)
    SES1 = pct_partner1 / 100.0
    SES2 = pct_partner2 / 100.0

    # Build feature matrix rows
    fm_header = [
        "Person",
        "Focus Duration on One Object (s)",
        "Gaze Shifts per Second",
        "% Time Looking at Workspace",
        "% Time Looking at Partner",
        "% Time Looking Elsewhere",
        "Average Gaze Direction (deg)",
        "Gaze Direction Variability (deg)",
        "Number of Fixations",
        "Average Focus Area Spread",
        "Time Both Look at the Same Region (s)",
        "Shared Attention Overlap Ratio",
        "Number of Shared Focus Episodes",
        "Average Duration per Shared Focus (s)",
        "Mutual Gaze Duration (s)",
        "Task Engagement Score",
        "Social Engagement Score"
    ]
    fm_rows = [
        [
            "P1",
            round(avg_fix1,4),
            round(shift_freq1,4),
            round(pct_workspace1,3),
            round(pct_partner1,3),
            round(pct_else1,3),
            round(avg_angle1,3),
            round(std_angle1,3),
            int(num_fix1),
            round(avg_spread1,6),
            round(joint_time_sec,4),
            round(mean_iou,6),
            int(num_episodes),
            round(avg_episode_dur,4),
            round(mutual_time,4),
            round(TES1,4),
            round(SES1,4)
        ],
        [
            "P2",
            round(avg_fix2,4),
            round(shift_freq2,4),
            round(pct_workspace2,3),
            round(pct_partner2,3),
            round(pct_else2,3),
            round(avg_angle2,3),
            round(std_angle2,3),
            int(num_fix2),
            round(avg_spread2,6),
            round(joint_time_sec,4),
            round(mean_iou,6),
            int(num_episodes),
            round(avg_episode_dur,4),
            round(mutual_time,4),
            round(TES2,4),
            round(SES2,4)
        ]
    ]
    # Save feature matrix
    fm_path = os.path.join(outdir, "p_17_feature_matrix.csv")
    with open(fm_path, "w", newline="") as f:
        w = csv.writer(f); w.writerow(fm_header); w.writerows(fm_rows)
    log(f"Saved feature_matrix: {fm_path}")

    # individual summary (first 10 columns)
    ind_header = fm_header[:10]
    ind_path = os.path.join(outdir, "p_17_gaze_individual_summary.csv")
    with open(ind_path, "w", newline="") as f:
        w = csv.writer(f); w.writerow(ind_header); w.writerows(fm_rows)
    log(f"Saved individual summary: {ind_path}")

    # pairwise summary
    pair_header = [
        "Time Both Look at the Same Region (s)",
        "Shared Attention Overlap Ratio",
        "Number of Shared Focus Episodes",
        "Average Duration per Shared Focus (s)",
        "Mutual Gaze Duration (s)"
    ]
    pair_row = [ round(joint_time_sec,4), round(mean_iou,6), int(num_episodes), round(avg_episode_dur,4), round(mutual_time,4) ]
    pair_path = os.path.join(outdir, "p_17_gaze_pairwise_summary.csv")
    with open(pair_path, "w", newline="") as f:
        w = csv.writer(f); w.writerow(pair_header); w.writerow(pair_row)
    log(f"Saved pairwise summary: {pair_path}")

    # Per-frame pair records (logs) — includes contexts and gaze points and inout
    records = []
    for i in range(n_frames):
        t = i / (fps + 1e-12)
        records.append({
            "frame": i,
            "time_s": round(t, 3),
            "P1_context": ctx1[i],
            "P2_context": ctx2[i],
            "P1_gaze_x": round(gp1_sm[i][0], 6) if gp1_sm[i] is not None else None,
            "P1_gaze_y": round(gp1_sm[i][1], 6) if gp1_sm[i] is not None else None,
            "P2_gaze_x": round(gp2_sm[i][0], 6) if gp2_sm[i] is not None else None,
            "P2_gaze_y": round(gp2_sm[i][1], 6) if gp2_sm[i] is not None else None,
            "P1_inout": rec1[i]["inout"],
            "P2_inout": rec2[i]["inout"],
            "joint_flag": bool(joint_flags[i]),
            "heatmap_iou": round(iou_list[i], 6)
        })
    recs_df = pd.DataFrame(records)
    recs_path = os.path.join(outdir, "p_17_pair_gaze_records.csv")
    recs_df.to_csv(recs_path, index=False)
    log(f"Saved per-frame pair records: {recs_path}")

    # Filtered event logs: moments when each person is looking_to_partner or looking_elsewhere
    def events_for_person(ctx_list, gp_sm, recs, person_tag):
        ev = []
        for i, ctx in enumerate(ctx_list):
            if ctx in ("looking_to_partner", "looking_elsewhere"):
                t = i / (fps + 1e-12)
                ev.append({
                    "frame": i,
                    "time_s": round(t, 3),
                    "context": ctx,
                    "gaze_x": round(gp_sm[i][0],6) if gp_sm[i] is not None else None,
                    "gaze_y": round(gp_sm[i][1],6) if gp_sm[i] is not None else None,
                    "inout": recs[i]["inout"],
                    "bbox": recs[i]["norm_bbox"]
                })
        return ev
    ev1 = events_for_person(ctx1, gp1_sm, rec1, "P1")
    ev2 = events_for_person(ctx2, gp2_sm, rec2, "P2")
    pd.DataFrame(ev1).to_csv(os.path.join(outdir, "p_17_p1_partner_elsewhere_events.csv"), index=False)
    pd.DataFrame(ev2).to_csv(os.path.join(outdir, "p_17_p2_partner_elsewhere_events.csv"), index=False)
    log(f"Saved per-person event files: p1_partner_elsewhere_events.csv, p2_partner_elsewhere_events.csv")

    # Annotated videos (draw heatmap + gaze point + label)
    log("Rendering annotated videos (this can take time)...")
    ann1 = []
    ann2 = []
    for i in range(n_frames):
        pil1 = Image.fromarray(frames1[i])
        pil2 = Image.fromarray(frames2[i])
        hm1_i = rec1[i]["heatmap"]
        hm2_i = rec2[i]["heatmap"]
        label1 = f"P1: {ctx1[i]}"
        label2 = f"P2: {ctx2[i]}"
        img1 = draw_overlay(pil1, rec1[i]["norm_bbox"], gp1_sm[i], hm=hm1_i, label=label1, color="lime")
        img2 = draw_overlay(pil2, rec2[i]["norm_bbox"], gp2_sm[i], hm=hm2_i, label=label2, color="cyan")
        ann1.append(img1)
        ann2.append(img2)
        if (i % 200) == 0 and i>0:
            log(f" Annotating frame {i}/{n_frames}...")

    out_v1 = os.path.join(outdir, "p_17_annotated_p1.mp4")
    out_v2 = os.path.join(outdir, "p_17_annotated_p2.mp4")
    # write as mp4
    iio.imwrite(out_v1, ann1, fps=fps, codec="libx264", quality=8)
    iio.imwrite(out_v2, ann2, fps=fps, codec="libx264", quality=8)
    log(f"Saved annotated videos:\n {out_v1}\n {out_v2}")

    return {
        "feature_matrix": fm_path,
        "individual_summary": ind_path,
        "pairwise_summary": pair_path,
        "pair_records": recs_path,
        "events_p1": os.path.join(outdir, "p_17_p1_partner_elsewhere_events.csv"),
        "events_p2": os.path.join(outdir, "p_17_p2_partner_elsewhere_events.csv"),
        "annotated_videos": (out_v1, out_v2)
    }

# -------------------- main -------------------------
if __name__ == "__main__":
    # sanity checks
    if not os.path.exists(OPENFACE_BIN):
        log(f"OpenFace binary not found at: {OPENFACE_BIN}")
        log("Please build OpenFace and set OPENFACE_BIN to your FeatureExtraction path.")
        sys.exit(1)
    if not os.path.exists(VIDEO1) or not os.path.exists(VIDEO2):
        log("Set VIDEO1/VIDEO2 paths at the top of the script and re-run.")
        sys.exit(1)
    os.makedirs(OUTDIR, exist_ok=True)
    t0 = time.time()
    res = compute_and_save_all(VIDEO1, VIDEO2, OUTDIR)
    t1 = time.time()
    log("All done.")
    log(f"Outputs: {res}")
    log(f"Elapsed: {t1 - t0:.1f} s")
